In [22]:
import math
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
import inspect

#--------------------------------------------------------
class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd) #bias ON
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1
        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        # not really a 'bias', more of a mask, but following the OpenAI/HF naming though
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)
        #calculate query, key, values for all heads in batch and move head forward to be the batch
        # nh is "number of heads", hs is "head size", and C (number of channels) = nh * hs
        # e.g. in GPT-2 (124M), n_head=12, hs=64, so nh*hs=C=768 channels in the Transformer
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        # attention (materializes the large (T,T) matrix for all the queries and keys)

        #flash attention v1 as T4 supports v1 only

        # att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        # att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
        # att = F.softmax(att, dim=-1)
        # y = att @ v  # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True)


        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
        # output projection
        y = self.c_proj(y)
        return y



class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x


class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x




@dataclass
class GPTConfig:
    block_size: int = 1024 # max sequence length
    vocab_size: int = 50257 # number of tokens: 50,000 BPE merges + 256 bytes tokens + 1 <|endoftext|> token
    n_layer: int = 12 # number of layers
    n_head: int = 12 # number of heads
    n_embd: int = 768 # embedding dimension


class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # weight sharing scheme
        self.transformer.wte.weight = self.lm_head.weight

        # init params
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, "NANOGPT_SCALE_INIT"):
                std *= 2 * (self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        # idx is of shape (B, T)
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size"
        # forward the token and position embeddings
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device).unsqueeze(0) # shape (T)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (T, n_embd)
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (B, T, n_embd)
        x = tok_emb + pos_emb
        # forward the blocks of the transformer
        for block in self.transformer.h:
            x = block(x)
        # forward the final layernorm and the classifer
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss


    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

    def configure_optimizers(self, weight_decay, learning_rate, device_type):
        # start with all of the candidate parameters (that require grad)
        param_dict = {pn: p for pn, p in self.named_parameters()}
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        # if master_process:
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == "cuda"
        # if master_process:
        print(f"using fused AdamW: {use_fused}")
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=(0.9, 0.95), eps=1e-8, fused=use_fused)
        return optimizer
# -----------------------------------------------------------------------------
import tiktoken


class DataLoaderLite:
    def __init__(self, B, T):
        self.B = B
        self.T = T

        # at init load tokens form disk and store them in memory
        with open('input.txt', 'r') as f:
            text = f.read()
        enc = tiktoken.get_encoding('gpt2')
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)
        print(f"loaded {len(tokens)} tokens")
        print(f"1 epoch = {len(self.tokens) // (B * T)} batches")

        #state
        self.current_position = 0

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position : self.current_position+B*T+1]
        x = (buf[:-1]).view(B, T) # inputs
        y = (buf[1:]).view(B, T) # outputs
        # advance the position in the tensor
        self.current_position += B * T
        # if loading the next batch would be out of bounds, reset
        if self.current_position + (B * T + 1) > len(self.tokens):
            self.current_position = 0
        return x, y

# -----------------------------------------------------------------------------


# Gradient Accumulation

In [26]:
# attempt to autodetect the device
import time
torch.backends.cuda.matmul.allow_tf32 = True  # no effect on T4, safe to enable
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision("high")


device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"using device: {device}")
# device = "cpu" #OVERRIDE
# max_length = 30
# num_return_sequences = 5

torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

total_batch_size = 524288 # 2**19, ~0.5M, in number of tokens
B = 8 # micro batch size
T = 1024 # sequence length
assert total_batch_size % (B * T) == 0, "make sure total_batch_size is divisible by B * T"
grad_accum_steps = total_batch_size // (B * T)
print(f"total desired batch size: {total_batch_size}")
print(f"calculated gradient accumulation steps: {grad_accum_steps}")

train_loader = DataLoaderLite(B=8, T=1024)


# get the logits
model = GPT(GPTConfig(vocab_size=50304))
# model = GPT.from_pretrained("gpt2")
model.to(device)

#compile the model
model = torch.compile(model)

max_lr = 6e-4
min_lr = max_lr * 0.1
warmup_steps = 10
max_steps = 50
def get_lr(it):
  # 1) linear warmup for warmup_iters steps
  if it < warmup_steps:
    return max_lr * (it+1) / warmup_steps
  # 2) if it > lr_decay_iters, return min learning rate
  if it >= max_steps:
    return min_lr
  # 3) in between, use consine decay down to min learning rate
  decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
  assert 0 <= decay_ratio <= 1
  coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges at 1 and goes to 0
  return min_lr + coeff * (max_lr - min_lr)

#optimize!
from torch import amp

scaler = amp.GradScaler(device="cuda")  # NEW: enable mixed precision

optimizer = model.configure_optimizers(weight_decay=0.1, learning_rate=6e-4, device_type=device)
# optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), eps=1e-8)

for step in range(max_steps):
    t0 = time.time()
    optimizer.zero_grad()
    loss_accum = 0.0

    #determine and set the learning rate for this iteration
    lr = get_lr(step)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    for micro_step in range(grad_accum_steps):
      x, y = train_loader.next_batch()
      x, y = x.to(device), y.to(device)

      # FP16 forward + loss (autocast enables mixed precision on T4)
      with amp.autocast("cuda"):
          logits, loss = model(x, y)

      # we have to scale the loss to account for gradient accumulation,
      # because the gradients just add on each successive backward().
      # addition of gradients corresponds to a SUM in the objective, but
      # instead of a SUM we want MEAN. Scale the loss here so it comes out right.
      loss = loss / grad_accum_steps # scale the loss
      loss_accum += loss.item()


      # backward pass (scaled for safe gradients in FP16)
      scaler.scale(loss).backward()

    #unscale before clipping
    scaler.unscale_(optimizer)
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)


    # update weights
    scaler.step(optimizer)
    scaler.update()

    torch.cuda.synchronize() # wait for GPU to finish work
    t1 = time.time()

    dt = (t1 - t0) # time difference
    ms = dt * 1000 #ms
    tokens_processed = train_loader.B * train_loader.T * grad_accum_steps
    tokens_per_sec = tokens_processed / dt
    print(
        f" step {step:4d} | loss: {loss_accum:.6f} | lr: {lr:.4e} | "
        f"norm: {norm:.4f} | dt: {ms:.2f}ms | tok/sec: {tokens_per_sec:.2f}"
     )


import sys; sys.exit(0)


#prefix tokens
import tiktoken
enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode("Hello, I'm a language model,")
tokens = torch.tensor(tokens, dtype=torch.long) #(8, )
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1) #(5, 8)
x = tokens.to(device)


# generate! right now x is (B, T) where B = 5, T = 8
# set the seed is 42 (my fav what is life :P)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
while x.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        logits = model(x) # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is  (5, 50)
        topk_probs, topk_indices = torch.topk(probs, k=50, dim=-1)
        # select a token from the top-k probabilities
        ix = torch.multinomial(topk_probs, num_samples=1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequnce
        x = torch.cat((x, xcol), dim=1)

# print the generated text
for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)


using device: cuda
total desired batch size: 524288
calculated gradient accumulation steps: 64
loaded 338025 tokens
1 epoch = 41 batches
num decayed parameter tensors: 50, with 124,354,560 parameters
num non-decayed parameter tensors: 98, with 121,344 parameters
using fused AdamW: True
 step    0 | loss: 10.961176 | lr: 6.0000e-05 | norm: 13.7307 | dt: 27031.35ms | tok/sec: 19395.55
 step    1 | loss: 9.763216 | lr: 1.2000e-04 | norm: 6.1150 | dt: 28921.67ms | tok/sec: 18127.86
 step    2 | loss: 9.339524 | lr: 1.8000e-04 | norm: 5.6734 | dt: 28643.70ms | tok/sec: 18303.78
 step    3 | loss: 9.667659 | lr: 2.4000e-04 | norm: 5.8280 | dt: 27450.20ms | tok/sec: 19099.61
 step    4 | loss: 9.066256 | lr: 3.0000e-04 | norm: 3.8648 | dt: 28097.71ms | tok/sec: 18659.46
 step    5 | loss: 8.615701 | lr: 3.6000e-04 | norm: 3.0571 | dt: 27388.44ms | tok/sec: 19142.68
 step    6 | loss: 8.372741 | lr: 4.2000e-04 | norm: 2.8869 | dt: 27243.26ms | tok/sec: 19244.69
 step    7 | loss: 8.053147 | lr

SystemExit: 0

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [27]:
# saving the model
torch.save(model.state_dict(), "gpt2_only_weights.pt")


# Batch Size Schedule + weight decay + FusedAdamW

In [21]:
# attempt to autodetect the device
import time
torch.backends.cuda.matmul.allow_tf32 = True  # no effect on T4, safe to enable
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision("high")


device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"using device: {device}")
# device = "cpu" #OVERRIDE
# max_length = 30
# num_return_sequences = 5

torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

train_loader = DataLoaderLite(B=8, T=1024)


# get the logits
model = GPT(GPTConfig(vocab_size=50304))
# model = GPT.from_pretrained("gpt2")
model.to(device)

#compile the model
model = torch.compile(model)

max_lr = 6e-4
min_lr = max_lr * 0.1
warmup_steps = 10
max_steps = 50
def get_lr(it):
  # 1) linear warmup for warmup_iters steps
  if it < warmup_steps:
    return max_lr * (it+1) / warmup_steps
  # 2) if it > lr_decay_iters, return min learning rate
  if it >= max_steps:
    return min_lr
  # 3) in between, use consine decay down to min learning rate
  decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
  assert 0 <= decay_ratio <= 1
  coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges at 1 and goes to 0
  return min_lr + coeff * (max_lr - min_lr)

#optimize!
from torch import amp

scaler = amp.GradScaler(device="cuda")  # NEW: enable mixed precision

optimizer = model.configure_optimizers(weight_decay=0.1, learning_rate=6e-4, device_type=device)
# optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), eps=1e-8)

for step in range(50):
    t0 = time.time()

    x, y = train_loader.next_batch()
    x, y = x.to(device), y.to(device)

    #determine and set the learning rate for this iteration
    lr = get_lr(step)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    optimizer.zero_grad()

    # FP16 forward + loss (autocast enables mixed precision on T4)
    with amp.autocast("cuda"):
        logits, loss = model(x, y)

    # backward pass (scaled for safe gradients in FP16)
    scaler.scale(loss).backward()

    #unscale before clipping
    scaler.unscale_(optimizer)
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)


    # update weights
    scaler.step(optimizer)
    scaler.update()

    torch.cuda.synchronize()
    t1 = time.time()

    dt = (t1 - t0) * 100  # time difference in milliseconds
    tokens_per_sec = (train_loader.B * train_loader.T) / (t1 - t0)
    print(f" step {step:4d} | loss: {loss.item():.6f} | lr: {lr:.4e} | norm: {norm:.4f} | dt: {dt:.2f}ms | tok/sec: {tokens_per_sec:.2f}")

import sys; sys.exit(0)


#prefix tokens
import tiktoken
enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode("Hello, I'm a language model,")
tokens = torch.tensor(tokens, dtype=torch.long) #(8, )
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1) #(5, 8)
x = tokens.to(device)


# generate! right now x is (B, T) where B = 5, T = 8
# set the seed is 42 (my fav what is life :P)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
while x.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        logits = model(x) # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is  (5, 50)
        topk_probs, topk_indices = torch.topk(probs, k=50, dim=-1)
        # select a token from the top-k probabilities
        ix = torch.multinomial(topk_probs, num_samples=1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequnce
        x = torch.cat((x, xcol), dim=1)

# print the generated text
for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)


using device: cuda
loaded 338025 tokens
1 epoch = 41 batches
num decayed parameter tensors: 50, with 124,354,560 parameters
num non-decayed parameter tensors: 98, with 121,344 parameters
using fused AdamW: True
 step    0 | loss: 10.945843 | lr: 6.0000e-05 | norm: 15.1871 | dt: 188.48ms | tok/sec: 4346.26
 step    1 | loss: 9.764231 | lr: 1.2000e-04 | norm: 6.3446 | dt: 40.91ms | tok/sec: 20022.32
 step    2 | loss: 9.106415 | lr: 1.8000e-04 | norm: 5.3951 | dt: 42.73ms | tok/sec: 19173.17
 step    3 | loss: 9.450719 | lr: 2.4000e-04 | norm: 6.0236 | dt: 41.80ms | tok/sec: 19597.15
 step    4 | loss: 8.929743 | lr: 3.0000e-04 | norm: 4.0037 | dt: 40.82ms | tok/sec: 20066.88
 step    5 | loss: 8.684343 | lr: 3.6000e-04 | norm: 2.6439 | dt: 42.13ms | tok/sec: 19444.11
 step    6 | loss: 8.620053 | lr: 4.2000e-04 | norm: 3.5140 | dt: 41.00ms | tok/sec: 19979.91
 step    7 | loss: 8.186541 | lr: 4.8000e-04 | norm: 2.2540 | dt: 41.50ms | tok/sec: 19738.48
 step    8 | loss: 7.806085 | lr: 5

SystemExit: 0

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Adding Learning Rate Schedular: Warmup + cosine decay (As instructed in GPT-3 Paper)

In [14]:
# attempt to autodetect the device
import time
torch.backends.cuda.matmul.allow_tf32 = True  # no effect on T4, safe to enable
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision("high")


device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"using device: {device}")
# device = "cpu" #OVERRIDE
# max_length = 30
# num_return_sequences = 5

torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

train_loader = DataLoaderLite(B=8, T=1024)


# get the logits
model = GPT(GPTConfig(vocab_size=50304))
# model = GPT.from_pretrained("gpt2")
model.to(device)

#compile the model
model = torch.compile(model)

max_lr = 6e-4
min_lr = max_lr * 0.1
warmup_steps = 10
max_steps = 50
def get_lr(it):
  # 1) linear warmup for warmup_iters steps
  if it < warmup_steps:
    return max_lr * (it+1) / warmup_steps
  # 2) if it > lr_decay_iters, return min learning rate
  if it >= max_steps:
    return min_lr
  # 3) in between, use consine decay down to min learning rate
  decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
  assert 0 <= decay_ratio <= 1
  coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges at 1 and goes to 0
  return min_lr + coeff * (max_lr - min_lr)

#optimize!
from torch import amp

scaler = amp.GradScaler(device="cuda")  # NEW: enable mixed precision

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), eps=1e-8)

for step in range(50):
    t0 = time.time()

    x, y = train_loader.next_batch()
    x, y = x.to(device), y.to(device)

    #determine and set the learning rate for this iteration
    lr = get_lr(step)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    optimizer.zero_grad()

    # FP16 forward + loss (autocast enables mixed precision on T4)
    with amp.autocast("cuda"):
        logits, loss = model(x, y)

    # backward pass (scaled for safe gradients in FP16)
    scaler.scale(loss).backward()

    #unscale before clipping
    scaler.unscale_(optimizer)
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)


    # update weights
    scaler.step(optimizer)
    scaler.update()

    torch.cuda.synchronize()
    t1 = time.time()

    dt = (t1 - t0) * 100  # time difference in milliseconds
    tokens_per_sec = (train_loader.B * train_loader.T) / (t1 - t0)
    print(f" step {step:4d} | loss: {loss.item():.6f} | lr: {lr:.4e} | norm: {norm:.4f} | dt: {dt:.2f}ms | tok/sec: {tokens_per_sec:.2f}")

import sys; sys.exit(0)


#prefix tokens
import tiktoken
enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode("Hello, I'm a language model,")
tokens = torch.tensor(tokens, dtype=torch.long) #(8, )
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1) #(5, 8)
x = tokens.to(device)


# generate! right now x is (B, T) where B = 5, T = 8
# set the seed is 42 (my fav what is life :P)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
while x.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        logits = model(x) # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is  (5, 50)
        topk_probs, topk_indices = torch.topk(probs, k=50, dim=-1)
        # select a token from the top-k probabilities
        ix = torch.multinomial(topk_probs, num_samples=1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequnce
        x = torch.cat((x, xcol), dim=1)

# print the generated text
for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)


using device: cuda
loaded 338025 tokens
1 epoch = 41 batches
 step    0 | loss: 10.945843 | lr: 6.0000e-05 | norm: 15.1871 | dt: 47.23ms | tok/sec: 17344.90
 step    1 | loss: 9.764231 | lr: 1.2000e-04 | norm: 6.3446 | dt: 42.49ms | tok/sec: 19277.70
 step    2 | loss: 9.106407 | lr: 1.8000e-04 | norm: 5.3955 | dt: 44.62ms | tok/sec: 18358.99
 step    3 | loss: 9.450704 | lr: 2.4000e-04 | norm: 6.0235 | dt: 42.35ms | tok/sec: 19343.18
 step    4 | loss: 8.929711 | lr: 3.0000e-04 | norm: 4.0037 | dt: 45.13ms | tok/sec: 18151.21
 step    5 | loss: 8.684285 | lr: 3.6000e-04 | norm: 2.6437 | dt: 43.62ms | tok/sec: 18779.22
 step    6 | loss: 8.620027 | lr: 4.2000e-04 | norm: 3.5158 | dt: 42.79ms | tok/sec: 19142.95
 step    7 | loss: 8.186493 | lr: 4.8000e-04 | norm: 2.2554 | dt: 43.34ms | tok/sec: 18899.58
 step    8 | loss: 7.806010 | lr: 5.4000e-04 | norm: 2.5879 | dt: 42.04ms | tok/sec: 19486.43
 step    9 | loss: 7.618584 | lr: 6.0000e-04 | norm: 2.3638 | dt: 43.83ms | tok/sec: 18692.

SystemExit: 0

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Added Gradient clipping (unscale before clipping) + Setting AdamW Hyperparameters as GPT-3 Paper

In [13]:
# attempt to autodetect the device
import time
torch.backends.cuda.matmul.allow_tf32 = True  # no effect on T4, safe to enable
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision("high")


device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"using device: {device}")
# device = "cpu" #OVERRIDE
# max_length = 30
# num_return_sequences = 5

torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

train_loader = DataLoaderLite(B=8, T=1024)


# get the logits
model = GPT(GPTConfig(vocab_size=50304))
# model = GPT.from_pretrained("gpt2")
model.to(device)

#compile the model
model = torch.compile(model)

#optimize!
from torch import amp

scaler = amp.GradScaler(device="cuda")  # NEW: enable mixed precision

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), eps=1e-8)

for i in range(50):
    t0 = time.time()

    x, y = train_loader.next_batch()
    x, y = x.to(device), y.to(device)

    optimizer.zero_grad()

    # FP16 forward + loss (autocast enables mixed precision on T4)
    with amp.autocast("cuda"):
        logits, loss = model(x, y)

    # backward pass (scaled for safe gradients in FP16)
    scaler.scale(loss).backward()

    #unscale before clipping
    scaler.unscale_(optimizer)
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update weights
    scaler.step(optimizer)
    scaler.update()

    torch.cuda.synchronize()
    t1 = time.time()

    dt = (t1 - t0) * 100  # time difference in milliseconds
    tokens_per_sec = (train_loader.B * train_loader.T) / (t1 - t0)
    print(f" step {i:4d} | loss: {loss.item():.6f} | norm: {norm:.4f} | dt: {dt:.2f}ms | tok/sec: {tokens_per_sec:.2f}")

import sys; sys.exit(0)


#prefix tokens
import tiktoken
enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode("Hello, I'm a language model,")
tokens = torch.tensor(tokens, dtype=torch.long) #(8, )
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1) #(5, 8)
x = tokens.to(device)


# generate! right now x is (B, T) where B = 5, T = 8
# set the seed is 42 (my fav what is life :P)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
while x.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        logits = model(x) # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is  (5, 50)
        topk_probs, topk_indices = torch.topk(probs, k=50, dim=-1)
        # select a token from the top-k probabilities
        ix = torch.multinomial(topk_probs, num_samples=1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequnce
        x = torch.cat((x, xcol), dim=1)

# print the generated text
for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)


using device: cuda
loaded 338025 tokens
1 epoch = 41 batches
 step    0 | loss: 10.945843 | norm: 15.1871 | dt: 49.95ms | tok/sec: 16400.08
 step    1 | loss: 9.607196 | norm: 4.7175 | dt: 45.34ms | tok/sec: 18069.54
 step    2 | loss: 8.898447 | norm: 4.4026 | dt: 47.75ms | tok/sec: 17155.79
 step    3 | loss: 8.856247 | norm: 4.1560 | dt: 44.61ms | tok/sec: 18363.70
 step    4 | loss: 8.624682 | norm: 3.7552 | dt: 46.28ms | tok/sec: 17699.27
 step    5 | loss: 8.443286 | norm: 3.0065 | dt: 45.10ms | tok/sec: 18166.00
 step    6 | loss: 8.361417 | norm: 2.0013 | dt: 45.66ms | tok/sec: 17939.72
 step    7 | loss: 8.069830 | norm: 2.7407 | dt: 45.26ms | tok/sec: 18100.41
 step    8 | loss: 7.767623 | norm: 2.1683 | dt: 45.56ms | tok/sec: 17980.40
 step    9 | loss: 7.696859 | norm: 1.4392 | dt: 45.85ms | tok/sec: 17865.52
 step   10 | loss: 7.678771 | norm: 1.9711 | dt: 44.91ms | tok/sec: 18241.73
 step   11 | loss: 7.481083 | norm: 1.9221 | dt: 46.50ms | tok/sec: 17617.73
 step   12 | 

SystemExit: 0

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Vocab size = 50304(nice number!!)  + Flash Attention + FP16 + grad scaler + AMP, batch_size=8, time:41ms

In [10]:
# attempt to autodetect the device
import time
torch.backends.cuda.matmul.allow_tf32 = True  # no effect on T4, safe to enable
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision("high")


device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"using device: {device}")
# device = "cpu" #OVERRIDE
# max_length = 30
# num_return_sequences = 5

torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

train_loader = DataLoaderLite(B=8, T=1024)


# get the logits
model = GPT(GPTConfig(vocab_size=50304))
# model = GPT.from_pretrained("gpt2")
model.to(device)

#compile the model
model = torch.compile(model)

#optimize!
from torch import amp

scaler = amp.GradScaler(device="cuda")  # NEW: enable mixed precision

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

for i in range(50):
    t0 = time.time()

    x, y = train_loader.next_batch()
    x, y = x.to(device), y.to(device)

    optimizer.zero_grad()

    # FP16 forward + loss (autocast enables mixed precision on T4)
    with amp.autocast("cuda"):
        logits, loss = model(x, y)

    # backward pass (scaled for safe gradients in FP16)
    scaler.scale(loss).backward()

    # update weights
    scaler.step(optimizer)
    scaler.update()

    torch.cuda.synchronize()
    t1 = time.time()

    dt = (t1 - t0) * 100  # time difference in milliseconds
    tokens_per_sec = (train_loader.B * train_loader.T) / (t1 - t0)
    print(f" step {i}, loss: {loss.item()}, dt: {dt:.2f}ms, tok/sec: {tokens_per_sec:.2f}")

import sys; sys.exit(0)


#prefix tokens
import tiktoken
enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode("Hello, I'm a language model,")
tokens = torch.tensor(tokens, dtype=torch.long) #(8, )
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1) #(5, 8)
x = tokens.to(device)


# generate! right now x is (B, T) where B = 5, T = 8
# set the seed is 42 (my fav what is life :P)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
while x.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        logits = model(x) # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is  (5, 50)
        topk_probs, topk_indices = torch.topk(probs, k=50, dim=-1)
        # select a token from the top-k probabilities
        ix = torch.multinomial(topk_probs, num_samples=1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequnce
        x = torch.cat((x, xcol), dim=1)

# print the generated text
for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)


using device: cuda
loaded 338025 tokens
1 epoch = 41 batches
 step 0, loss: 10.945842742919922, dt: 49.25ms, tok/sec: 16633.03
 step 1, loss: 9.606761932373047, dt: 43.78ms, tok/sec: 18712.09
 step 2, loss: 8.932099342346191, dt: 45.56ms, tok/sec: 17981.86
 step 3, loss: 8.75814151763916, dt: 43.63ms, tok/sec: 18776.98
 step 4, loss: 8.625509262084961, dt: 44.42ms, tok/sec: 18440.69
 step 5, loss: 8.481952667236328, dt: 43.89ms, tok/sec: 18666.34
 step 6, loss: 8.383893966674805, dt: 43.87ms, tok/sec: 18671.73
 step 7, loss: 8.065706253051758, dt: 44.32ms, tok/sec: 18484.42
 step 8, loss: 7.807280540466309, dt: 43.96ms, tok/sec: 18636.41
 step 9, loss: 7.785613059997559, dt: 43.69ms, tok/sec: 18751.84
 step 10, loss: 7.750455856323242, dt: 43.48ms, tok/sec: 18841.01
 step 11, loss: 7.555953025817871, dt: 43.11ms, tok/sec: 19003.71
 step 12, loss: 7.504838943481445, dt: 43.99ms, tok/sec: 18624.47
 step 13, loss: 7.300006866455078, dt: 42.99ms, tok/sec: 19053.48
 step 14, loss: 7.1924552

SystemExit: 0

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Flash Attention + FP16 + grad scaler + AMP

In [7]:
# attempt to autodetect the device
import time
torch.backends.cuda.matmul.allow_tf32 = True  # no effect on T4, safe to enable
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision("high")


device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"using device: {device}")
# device = "cpu" #OVERRIDE
# max_length = 30
# num_return_sequences = 5

torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

train_loader = DataLoaderLite(B=8, T=1024)


# get the logits
model = GPT(GPTConfig())
# model = GPT.from_pretrained("gpt2")
model.to(device)

#compile the model
model = torch.compile(model)

#optimize!
from torch import amp

scaler = amp.GradScaler(device="cuda")  # NEW: enable mixed precision

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

for i in range(50):
    t0 = time.time()

    x, y = train_loader.next_batch()
    x, y = x.to(device), y.to(device)

    optimizer.zero_grad()

    # FP16 forward + loss (autocast enables mixed precision on T4)
    with amp.autocast("cuda"):
        logits, loss = model(x, y)

    # backward pass (scaled for safe gradients in FP16)
    scaler.scale(loss).backward()

    # update weights
    scaler.step(optimizer)
    scaler.update()

    torch.cuda.synchronize()
    t1 = time.time()

    dt = (t1 - t0) * 100  # time difference in milliseconds
    tokens_per_sec = (train_loader.B * train_loader.T) / (t1 - t0)
    print(f" step {i}, loss: {loss.item()}, dt: {dt:.2f}ms, tok/sec: {tokens_per_sec:.2f}")

import sys; sys.exit(0)


#prefix tokens
import tiktoken
enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode("Hello, I'm a language model,")
tokens = torch.tensor(tokens, dtype=torch.long) #(8, )
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1) #(5, 8)
x = tokens.to(device)


# generate! right now x is (B, T) where B = 5, T = 8
# set the seed is 42 (my fav what is life :P)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
while x.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        logits = model(x) # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is  (5, 50)
        topk_probs, topk_indices = torch.topk(probs, k=50, dim=-1)
        # select a token from the top-k probabilities
        ix = torch.multinomial(topk_probs, num_samples=1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequnce
        x = torch.cat((x, xcol), dim=1)

# print the generated text
for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)


using device: cuda
loaded 338025 tokens
1 epoch = 41 batches
 step 0, loss: 10.914106369018555, dt: 3179.67ms, tok/sec: 257.64
 step 1, loss: 9.548101425170898, dt: 47.07ms, tok/sec: 17402.41
 step 2, loss: 8.843807220458984, dt: 44.54ms, tok/sec: 18391.02
 step 3, loss: 8.736413955688477, dt: 43.03ms, tok/sec: 19038.46
 step 4, loss: 8.498735427856445, dt: 43.59ms, tok/sec: 18792.44
 step 5, loss: 8.36872673034668, dt: 43.57ms, tok/sec: 18802.49
 step 6, loss: 8.34364128112793, dt: 43.35ms, tok/sec: 18895.53
 step 7, loss: 8.049447059631348, dt: 43.80ms, tok/sec: 18702.10
 step 8, loss: 7.739285469055176, dt: 42.89ms, tok/sec: 19098.87
 step 9, loss: 7.676781177520752, dt: 44.46ms, tok/sec: 18427.19
 step 10, loss: 7.6713643074035645, dt: 43.55ms, tok/sec: 18809.93
 step 11, loss: 7.512412071228027, dt: 44.49ms, tok/sec: 18411.07
 step 12, loss: 7.449471473693848, dt: 43.36ms, tok/sec: 18894.11
 step 13, loss: 7.239461898803711, dt: 43.42ms, tok/sec: 18867.24
 step 14, loss: 7.1308498

SystemExit: 0

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# FP16 + grad scaler + AMP

In [5]:
# attempt to autodetect the device
import time
torch.backends.cuda.matmul.allow_tf32 = True  # no effect on T4, safe to enable
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision("high")


device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"using device: {device}")
# device = "cpu" #OVERRIDE
# max_length = 30
# num_return_sequences = 5

torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

train_loader = DataLoaderLite(B=8, T=1024)


# get the logits
model = GPT(GPTConfig())
# model = GPT.from_pretrained("gpt2")
model.to(device)

#compile the model
model = torch.compile(model)

#optimize!
from torch import amp

scaler = amp.GradScaler(device="cuda")  # NEW: enable mixed precision

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

for i in range(50):
    t0 = time.time()

    x, y = train_loader.next_batch()
    x, y = x.to(device), y.to(device)

    optimizer.zero_grad()

    # FP16 forward + loss (autocast enables mixed precision on T4)
    with amp.autocast("cuda"):
        logits, loss = model(x, y)

    # backward pass (scaled for safe gradients in FP16)
    scaler.scale(loss).backward()

    # update weights
    scaler.step(optimizer)
    scaler.update()

    torch.cuda.synchronize()
    t1 = time.time()

    dt = (t1 - t0) * 100  # time difference in milliseconds
    tokens_per_sec = (train_loader.B * train_loader.T) / (t1 - t0)
    print(f" step {i}, loss: {loss.item()}, dt: {dt:.2f}ms, tok/sec: {tokens_per_sec:.2f}")

import sys; sys.exit(0)


#prefix tokens
import tiktoken
enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode("Hello, I'm a language model,")
tokens = torch.tensor(tokens, dtype=torch.long) #(8, )
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1) #(5, 8)
x = tokens.to(device)


# generate! right now x is (B, T) where B = 5, T = 8
# set the seed is 42 (my fav what is life :P)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
while x.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        logits = model(x) # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is  (5, 50)
        topk_probs, topk_indices = torch.topk(probs, k=50, dim=-1)
        # select a token from the top-k probabilities
        ix = torch.multinomial(topk_probs, num_samples=1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequnce
        x = torch.cat((x, xcol), dim=1)

# print the generated text
for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)


using device: cuda
loaded 338025 tokens
1 epoch = 41 batches
 step 0, loss: 10.914105415344238, dt: 5613.79ms, tok/sec: 145.93
 step 1, loss: 9.548095703125, dt: 58.02ms, tok/sec: 14119.42
 step 2, loss: 8.84383773803711, dt: 55.70ms, tok/sec: 14707.44
 step 3, loss: 8.736400604248047, dt: 54.90ms, tok/sec: 14922.54
 step 4, loss: 8.498760223388672, dt: 54.49ms, tok/sec: 15033.17
 step 5, loss: 8.368730545043945, dt: 54.69ms, tok/sec: 14979.76
 step 6, loss: 8.343643188476562, dt: 54.67ms, tok/sec: 14984.44
 step 7, loss: 8.049449920654297, dt: 54.55ms, tok/sec: 15016.63
 step 8, loss: 7.739282608032227, dt: 54.65ms, tok/sec: 14991.04
 step 9, loss: 7.67677116394043, dt: 54.70ms, tok/sec: 14977.12
 step 10, loss: 7.671354293823242, dt: 54.72ms, tok/sec: 14971.85
 step 11, loss: 7.512383460998535, dt: 54.59ms, tok/sec: 15007.05
 step 12, loss: 7.449495792388916, dt: 54.70ms, tok/sec: 14976.69
 step 13, loss: 7.239490509033203, dt: 54.74ms, tok/sec: 14964.50
 step 14, loss: 7.13085031509

SystemExit: 0

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
